# Install Requirements

In [ ]:
%pip install -r requirements.txt

# Dependencies

In [1]:
import pandas as pd
import numpy as np
import os, requests, json, csv
import psycopg2
from dotenv import load_dotenv

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9156\3630013386.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
from pydantic import BaseModel, Field
from openai import OpenAI

# Initialize Ontology Translator

In [ ]:
# Function to load ontology into memory
def load_ontology_from_db() -> dict:
    conn = psycopg2.connect(os.getenv("DATABASE_URL"))
    cur = conn.cursor()

    ontology_data = {
        "uberon-tissue": {},
        "cl-cell": {},
        "ensembl-gene": {}
    }

    table_mappings = [
        ("TISSUE", "UBERON", "tissueName", "uberon-tissue"),
        ("CELL", "CL", "cellName", "cl-cell"),
        ("GENE", "ENSEMBL", "geneName", "ensembl-gene")
    ]

    for table, key_col, value_col, json_key in table_mappings:
        cur.execute(f"SELECT {key_col}, {value_col} FROM {table}")
        rows = cur.fetchall()
        ontology_data[json_key] = {key: value for key, value in rows}

    cur.close()
    conn.close()

    return ontology_data

In [2]:
import psycopg2
import os
from contextlib import closing

def load_ontology_from_db() -> dict:
    ontology_data = {
        "uberon-tissue": {},
        "cl-cell": {},
        "ensembl-gene": {}
    }

    table_mappings = [
        ("TISSUE", "UBERON", "tissueName", "uberon-tissue"),
        ("CELL", "CL", "cellName", "cl-cell"),
        ("GENE", "ENSEMBL", "geneName", "ensembl-gene")
    ]

    query = " UNION ALL ".join(
        f"SELECT '{json_key}' AS type, {key_col} AS key, {value_col} AS value FROM {table}"
        for table, key_col, value_col, json_key in table_mappings
    )

    with closing(psycopg2.connect(os.getenv("DATABASE_URL"))) as conn, conn.cursor() as cur:
        cur.execute(query)
        for row in cur.fetchall():
            json_key, key, value = row
            ontology_data[json_key][key] = value

    return ontology_data

In [3]:
class OntologyTranslator:
    _ontology_dict = None  # Class-level cache

    @classmethod
    def _load_ontology(cls):
        if cls._ontology_dict is None:
            cls._ontology_dict = load_ontology_from_db()

    @classmethod
    def tissue(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("uberon-tissue", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def cell(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("cl-cell", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def gene(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("ensembl-gene", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

In [4]:
ontl = OntologyTranslator

# Load CSV

In [5]:
def csv_read(file_path):
    with open(file_path, 'r') as file:
        sample = file.read(1024) 
        file.seek(0)
        detected_delimiter = csv.Sniffer().sniff(sample).delimiter    
    # Read the file with the detected delimiter
    df = pd.read_csv(file_path, sep=detected_delimiter)
    return df

In [6]:
file_path = "cluster0.csv"
df = csv_read(file_path)

In [ ]:
df

# Preprocess Data

## Group Genes by Clusters

In [7]:
def extract_clusters(df):
    df.columns = df.columns.str.lower()
    # Ensure the column names are correct
    if "cluster" not in df.columns or "gene" not in df.columns:
        raise ValueError("The file must contain 'Cluster' and 'Gene' columns")

    # Group the data by clusters and pad genes to ensure columns have equal lengths
    grouped = df.groupby("cluster")["gene"].apply(list)

    # Create a DataFrame where each column is a cluster and rows contain the genes
    max_length = max(grouped.apply(len))  # Determine the maximum number of genes in a cluster
    return pd.DataFrame({cluster: genes + [None] * (max_length - len(genes)) for cluster, genes in grouped.items()})

In [8]:
clustered_df = extract_clusters(df)

clustered_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,KRT14,TXNIP,LY6D,S100A8,TIMP1,ITM2B,TINAGL1,KRT1,KRT16,KRT14,...,IGHG4,FADS2,IGKC,SPRR1B,IGHG4,FASN,LOR,S100A8,COL3A1,COL1A1
1,IFI27,ATP1B3,KRT14,S100A7,APOD,PERP,TM4SF1,GPX2,SBSN,LY6D,...,IGKC,THRSP,IGHG4,S100A7,IGKC,APOC1,FLG2,S100A7,COL1A1,COL3A1
2,COL17A1,MAF,KRT16,SPRR1B,SAA1,GPNMB,IGFBP6,LTF,LY6D,S100A2,...,MYL9,FADS1,IGHG3,SPRR2D,MALAT1,MGST1,ARG1,CSTA,COL1A2,SPARC
3,HSPA1A,EIF4A2,LGALS7,KRTDAP,IGFBP5,TXNIP,MMP28,CCL27,LGALS7,LGALS7,...,IGHG3,PECR,IGHG1,MUCL1,C1QA,SAA1,LCE1B,KRTDAP,SPARC,MT-CYB
4,LGALS3BP,MMP13,IFI27,SBSN,C3,GJB6,BIRC5,FOSB,KRTDAP,IFI27,...,IGHG1,HSD11B1,JCHAIN,S100A7A,HLA-DRA,AWAT2,KPRP,SPRR1B,POSTN,MT-ND2
5,EFNA1,CAPN2,S100A2,CRABP2,MMP3,ATP1B3,CA2,EPHB6,DMKN,KRT5,...,JCHAIN,HSD3B1,IGHA1,SPRR1A,MT2A,ACSBG1,WFDC12,GJB6,DCN,None
6,RBM42,SULF2,HIST1H1C,SLPI,COMP,CLDN1,CENPW,ATF3,GJB2,TAGLN2,...,IGFBP6,FABP7,IGLC2,IL36RN,MYL9,CYP4F8,LCE1A,MUCL1,COL6A3,None
7,HIST1H1C,CEBPG,LSR,IVL,CXCL12,TNFSF10,SERPINA3,TNFRSF19,HIST1H1C,FXYD3,...,SULF1,ACSM6,IGLC1,LCE3E,IGHG1,TLCD4,LCE2C,SLPI,COL5A2,None
8,SCPEP1,PALLD,FAM83H,PI3,CCL19,DSG3,RRM2,AC103591.3,CRABP2,HSPB1,...,IGHA1,ACOT1,IGHG2,SLC6A14,ATP6V0C,CLMP,LCE2B,S100A7A,PRRX1,None
9,GAMT,PLAU,TMEM79,MUCL1,MMP1,MAF,CCNB2,HLA-DQB2,SLPI,COX6B1,...,GGT5,PKLR,MZB1,SPP1,IGHG3,GLDC,LCE1E,SPP1,COL11A1,None


## Translate Genes into Ensembl ID

In [9]:
def translate_genes(clustered_df):
    # Translate gene names to Ensembl IDs in the clustered DataFrame
    return clustered_df.apply(lambda col: col.map(lambda gene: ontl.gene(gene, rev=True) if pd.notna(gene) else None))

In [15]:
ensembl_df = translate_genes(clustered_df)
ensembl_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,ENSG00000186847,ENSG00000265972,ENSG00000167656,ENSG00000143546,ENSG00000102265,ENSG00000136156,ENSG00000142910,ENSG00000167768,ENSG00000186832,ENSG00000186847,...,ENSG00000211892,ENSG00000134824,ENSG00000211592,ENSG00000169469,ENSG00000211892,ENSG00000169710,None,ENSG00000143546,ENSG00000168542,ENSG00000108821
1,ENSG00000165949,ENSG00000069849,ENSG00000186847,ENSG00000143556,ENSG00000189058,ENSG00000112378,ENSG00000169908,ENSG00000176153,ENSG00000189001,ENSG00000167656,...,ENSG00000211592,ENSG00000151365,ENSG00000211892,ENSG00000143556,ENSG00000211592,ENSG00000130208,ENSG00000143520,ENSG00000143556,ENSG00000108821,ENSG00000168542
2,ENSG00000065618,ENSG00000178573,ENSG00000186832,ENSG00000169469,ENSG00000173432,ENSG00000136235,ENSG00000167779,ENSG00000012223,ENSG00000167656,ENSG00000196754,...,ENSG00000101335,ENSG00000149485,ENSG00000211897,ENSG00000163216,ENSG00000251562,ENSG00000008394,ENSG00000118520,ENSG00000121552,ENSG00000164692,ENSG00000113140
3,ENSG00000204389,ENSG00000156976,ENSG00000205076,ENSG00000188508,ENSG00000115461,ENSG00000265972,ENSG00000271447,ENSG00000213927,ENSG00000205076,ENSG00000205076,...,ENSG00000211897,ENSG00000115425,ENSG00000211896,ENSG00000172551,ENSG00000173372,ENSG00000173432,ENSG00000196734,ENSG00000188508,ENSG00000113140,ENSG00000198727
4,ENSG00000108679,ENSG00000137745,ENSG00000165949,ENSG00000189001,None,ENSG00000121742,ENSG00000089685,ENSG00000125740,ENSG00000188508,ENSG00000165949,...,ENSG00000211896,ENSG00000117594,ENSG00000132465,ENSG00000184330,ENSG00000204287,ENSG00000147160,ENSG00000203786,ENSG00000169469,ENSG00000133110,ENSG00000198763
5,ENSG00000169242,ENSG00000162909,ENSG00000196754,ENSG00000143320,ENSG00000149968,ENSG00000069849,ENSG00000104267,ENSG00000106123,ENSG00000161249,ENSG00000186081,...,ENSG00000132465,ENSG00000203857,ENSG00000211895,ENSG00000169474,ENSG00000125148,ENSG00000103740,ENSG00000168703,ENSG00000121742,ENSG00000011465,None
6,ENSG00000126254,ENSG00000196562,None,ENSG00000124107,ENSG00000105664,ENSG00000163347,ENSG00000203760,ENSG00000162772,ENSG00000165474,ENSG00000158710,...,ENSG00000167779,ENSG00000164434,ENSG00000211677,ENSG00000136695,ENSG00000101335,ENSG00000186526,ENSG00000186844,ENSG00000172551,ENSG00000163359,None
7,None,ENSG00000153879,ENSG00000105699,ENSG00000163207,ENSG00000107562,ENSG00000121858,ENSG00000196136,ENSG00000127863,None,ENSG00000089356,...,ENSG00000137573,ENSG00000173124,ENSG00000211675,ENSG00000185966,ENSG00000211896,ENSG00000152078,ENSG00000187180,ENSG00000124107,ENSG00000204262,None
8,ENSG00000121064,ENSG00000129116,ENSG00000180921,ENSG00000124102,ENSG00000172724,ENSG00000134757,ENSG00000171848,None,ENSG00000143320,ENSG00000106211,...,ENSG00000211895,ENSG00000184227,ENSG00000211893,ENSG00000268104,ENSG00000185883,ENSG00000166250,ENSG00000159455,ENSG00000184330,ENSG00000116132,None
9,ENSG00000130005,ENSG00000122861,ENSG00000163472,ENSG00000172551,ENSG00000196611,ENSG00000178573,ENSG00000157456,ENSG00000232629,ENSG00000124107,ENSG00000126267,...,ENSG00000099998,ENSG00000143627,ENSG00000170476,ENSG00000118785,ENSG00000211897,ENSG00000178445,ENSG00000186226,ENSG00000118785,ENSG00000060718,None


# Statistical Analysis with CellxGene's Gene Expression Data

## Function Initialization

Function to fetch gene expression data

In [ ]:
def fetch_expression_data(cluster, ensembl_ids):
    """
    Fetches gene expression data from the Cellxgene API for a given cluster and list of Ensembl IDs.
    
    :param cluster: The cluster name.
    :param ensembl_ids: List of Ensembl IDs for the cluster.
    :return: JSON response from the API or None if an error occurred.
    """

    payload = {
        "filter": {
            "dataset_ids": [],
            "development_stage_ontology_term_ids": [],
            "disease_ontology_term_ids": [],
            "gene_ontology_term_ids": ensembl_ids,
            "organism_ontology_term_id": "NCBITaxon:9606",
            "self_reported_ethnicity_ontology_term_ids": [],
            "sex_ontology_term_ids": [],
            "publication_citations": [],
        },
        "is_rollup": True
    }

    # URL for the POST request
    API_URL = "https://api.cellxgene.cziscience.com/wmg/v2/query"

    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()['expression_summary']
    except requests.RequestException as e:
        print(f"Error fetching data for Cluster {cluster}: {e}")
        return None

Function to convert the data to a DataFrame

In [ ]:
def expression_data_to_df(data):
    flattened_data = []
    for gene_id, anatomical_structures in data.items():
        for anatomical_id, cell_types in anatomical_structures.items():
            for cell_type_id, aggregated_data in cell_types.items():
                metrics = aggregated_data['aggregated']
                flattened_data.append({
                    'gene': gene_id,
                    'tissue': anatomical_id,
                    'cell': cell_type_id,
                    'expression': metrics.get('me', None),
                    'cell count': metrics.get('n', None),
                    'cell percentage': metrics.get('pc', None),
                    'tissue composition': metrics.get('tpc', None)
                })
    return pd.DataFrame(flattened_data)

Function to filter the data

In [ ]:
def filter_expression_data(response_df, TARGET_UBERON_ID):
    tissue_df = response_df[response_df['tissue'] == TARGET_UBERON_ID].drop(columns=['tissue'])
    return tissue_df[~tissue_df['cell'].isin(['tissue_stats', 'CL:0000000'])]

Function to translate the ontology terms

In [ ]:
def translate_ontology(df):
    df['cell'] = df['cell'].apply(lambda x: ontl.cell(x))
    df['gene'] = df['gene'].apply(lambda x: ontl.gene(x)).fillna(df['gene'])

    return df

In [ ]:
def calculate_cell_score(results):
    cell_df = results.copy()

    # Group by 'cell' and calculate the total cell count for each group
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    cell_df = cell_df[cell_df['total cell count'] >= 100]

    # Calculate scores
    cell_df['score'] = (1) * ((cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())) + (1.5) * (cell_df['cell percentage']) + (2.5) * ((cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min()))

    # Group by 'cell' and calculate the standard deviation of 'score' for each group
    std_scores = cell_df.groupby('cell')['score'].std().reset_index()
    std_scores.columns = ['cell', 'std']
    cell_df = cell_df.merge(std_scores, on='cell')

    # Group by 'cell' and calculate the mean of 'score' for each group
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().fillna(0)
    mean_scores.columns = ['cell', 'mean']
    cell_df = cell_df.merge(mean_scores, on='cell')

    # Calculate the coefficient of variation (CV) for each cell
    cell_df['CV'] = cell_df['std'] / cell_df['mean']

    # Group by 'cell' and calculate the kurtosis of 'score' for each group
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index()
    kurtosis_scores.columns = ['cell', 'kurtosis']
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    # Normalize the scores
    cell_df['std norm'] = (cell_df['std'] - cell_df['std'].min()) / (cell_df['std'].max() - cell_df['std'].min())
    cell_df['mean norm'] = (cell_df['mean'] - cell_df['mean'].min()) / (cell_df['mean'].max() - cell_df['mean'].min())
    cell_df['CV norm'] = (cell_df['CV'] - cell_df['CV'].min()) / (cell_df['CV'].max() - cell_df['CV'].min())
    cell_df['kurtosis norm'] = ((cell_df['kurtosis'] - cell_df['kurtosis'].min()) / (cell_df['kurtosis'].max() - cell_df['kurtosis'].min()))

    # Count the number of entries for each cell
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    cell_df['cell score'] = (0.1 * (1 - cell_df['std norm']).fillna(0) + 0.7 * cell_df['mean norm'] + 0.1 * (1 - cell_df['CV norm']) + 0.1 * (1 - cell_df['kurtosis norm']).fillna(0)) * cell_df['count']
    return cell_df.sort_values(by=['count', 'cell score', 'cell', 'gene'], ascending=[False, False, False, True]).reset_index(drop=True)

In [ ]:
def calculate_cell_score(
    results,
    w_e=1.0, w_p=1.5, w_ct=2.5,             # base score weights
    w_m=7.0, w_f=3.0,                       # mean vs flatness weights
    alpha=1.0, beta=1.0, gamma=1.0,         # flatness breakdown
    w_c=1.0                                 # count weight
):
    import pandas as pd
    import numpy as np

    cell_df = results.copy()

    # Calculate total cell count per cell
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    # Adaptive cutoff: max(μ - 0.5σ, min(total count))
    mu = cell_df['total cell count'].mean()
    sigma = cell_df['total cell count'].std()
    minimum = cell_df['total cell count'].min()
    min_count = max(mu - 0.5 * sigma, minimum)
    cell_df = cell_df[cell_df['total cell count'] >= min_count]

    # Normalize expression and cell count
    norm_expr = (cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())
    norm_count = (cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min())

    # Compute base score
    cell_df['score'] = (
        w_e * norm_expr +
        w_p * cell_df['cell percentage'] +
        w_ct * norm_count
    )

    # Compute flatness components
    std_scores = cell_df.groupby('cell')['score'].std().reset_index().rename(columns={'score': 'std'})
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().rename(columns={'score': 'mean'})
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index(name='kurtosis')

    cell_df = cell_df.merge(std_scores, on='cell')
    cell_df = cell_df.merge(mean_scores, on='cell')
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    cell_df['CV'] = cell_df['std'] / (cell_df['mean'].replace(0, np.nan))

    # Normalize flatness metrics
    for col in ['std', 'CV', 'kurtosis', 'mean']:
        col_min, col_max = cell_df[col].min(), cell_df[col].max()
        cell_df[f'{col} norm'] = (cell_df[col] - col_min) / (col_max - col_min + 1e-9)

    # Stability (flatness) score
    stability = (
        alpha * (1 - cell_df['std norm'].fillna(0)) +
        beta * (1 - cell_df['CV norm'].fillna(0)) +
        gamma * (1 - cell_df['kurtosis norm'].fillna(0))
    )
    flatness_weight = stability / (alpha + beta + gamma)

    # Count-based log-scaled soft weight
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    log_scaled_count = np.log1p(cell_df['count']) / np.log1p(cell_df['count'].max())

    # Final cell score
    cell_df['cell score'] = (
        (w_m * cell_df['mean norm'] + w_f * flatness_weight) *
        (1 + w_c * log_scaled_count)
    )

    return cell_df.sort_values(by=['count', 'cell score', 'cell', 'gene'], ascending=[False, False, False, True]).reset_index(drop=True)


In [ ]:
def run_data_processing(cluster, ensembl_ids, target_uberon_id):
    try:
        data = fetch_expression_data(cluster, ensembl_ids)

        response_df = expression_data_to_df(data)

        filtered_df = filter_expression_data(response_df, target_uberon_id)

        translated_df = translate_ontology(filtered_df)

        ranked_df = calculate_cell_score(translated_df)

        return ranked_df

    except Exception as e:
        print(f"An error occurred during the pipeline execution: {e}")
        return pd.DataFrame()

In [ ]:
def main_data_analysis(ensembl_df, target_uberon_id):
    results = {}

    # Iterate over each cluster (column) in the DataFrame
    for cluster in ensembl_df.columns:
        # Extract Ensembl IDs for the current cluster, dropping any NaN values
        ensembl_ids = ensembl_df[cluster].dropna().tolist()
        
        if not ensembl_ids:
            continue
        
        # Run the expression pipeline for the current cluster
        result_df = run_data_processing(cluster, ensembl_ids, target_uberon_id)
        
        # Store the result in the dictionary
        results[cluster] = result_df.reset_index(drop=True)

    return results

Function to Display the Results

In [ ]:
from IPython.display import Markdown, display
def display_dataframe_as_markdown(df):
    markdown_table = df.to_markdown(index=False)
    display(Markdown(markdown_table))

## Run the Section

In [ ]:
# Specify the UBERON ID to filter
TARGET_UBERON_ID = "UBERON:0002097"

In [ ]:
# Run the main pipeline
results = main_data_analysis(ensembl_df, TARGET_UBERON_ID)

In [ ]:
display_dataframe_as_markdown(results[1])

In [ ]:
results[0]['cell'].unique()[:5]

In [ ]:
ex_df = results[0].copy()
unique_cells = ex_df['cell'].unique()[:10]

ex_df = ex_df[ex_df['cell'].isin(unique_cells)][['cell', 'cell score']].drop_duplicates().reset_index(drop=True)
ex_df

In [ ]:
results[0][results[0]['cell']=='epithelial cell'][['gene', 'expression', 'cell count', 'cell percentage', 'tissue composition', 'score']]

In [ ]:
def export_dataframe(df, n=10):
    unique_cells = df['cell'].unique()[:n]
    filtered_df = df[df['cell'].isin(unique_cells)]

    final_df = (
        filtered_df.groupby('cell', as_index=False)
        .agg({'gene': lambda x: ", ".join(map(str, set(x))), 
              'cell score': lambda x: ", ".join(map(str, set(x)))})
    ).sort_values(by=['cell score'], ascending=[False])

    return final_df

In [ ]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

In [ ]:
def get_top3_cells(df):
    return df['cell'].unique()[:3]

In [ ]:
def export_top3_cells(results):

    top3cells = pd.DataFrame()
    for cluster in results:
        top3cells[f'{cluster}'] = get_top3_cells(results[cluster])

    top3cellsT = top3cells.T.reset_index()
    top3cellsT.columns = ['Cluster'] + [f'Cell {i+1}' for i in range(top3cellsT.shape[1]-1)]

    return top3cellsT

In [ ]:
display_dataframe_as_markdown(export_top3_cells(results))

# Validation with Tavily and Gemini Connected to a Database

In [ ]:
import psycopg2
import os
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

In [ ]:
# Load environment variables
load_dotenv(override=True)

# Initialize Tavily and Gemini clients
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(os.getenv("DATABASE_URL"))

In [ ]:
def check_database(tissue_name, cell_name, gene_name):
    query = """
        SELECT relationcertainty FROM gene_expression
        WHERE uberon = (
            SELECT uberon 
            FROM tissue 
            WHERE tissuename = %s
        )
        AND cl = (
            SELECT cl 
            FROM cell 
            WHERE cellname = %s
        )
        AND ensembl = (
            SELECT ensembl 
            FROM gene 
            WHERE genename = %s
        )
    """
    try:
        with get_db_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(query, (tissue_name, cell_name, gene_name))
                result = cur.fetchone()
                return result[0] if result else None
    except Exception as e:
        print(f"Database error: {e}")
        return None

In [ ]:
# Function to query Tavily using SDK
def query_tavily(tissue_name, cell_name, gene_name):
    query = f"Does {cell_name} express {gene_name} in {tissue_name}?"
    try:
        response = tavily_client.search(query= query, search_depth= "basic")
        sources = [result["url"] for result in response.get("results", [])]
        return sources if sources else None
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

In [ ]:
# Function to summarize using OpenAI
def summarize_with_openai(tissue_name, cell_name, gene_name, sources):
    prompt = f"""
    Given the following sources, determine the confidence level that {cell_name} expresses {gene_name} in {tissue_name}.
    Return one of these values in valid JSON format:
    {{"status": 1.0}} → Yes
    {{"status": 0.67}} → Mostly Yes
    {{"status": 0.33}} → Mostly No
    {{"status": 0.0}} → No
    never return with markdown formatting.
    Sources: {sources}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are an expert in cellular biology."},
                      {"role": "user", "content": prompt}]
        )
        output = response.choices[0].message.content
        return json.loads(output)["status"]
    except Exception as e:
        print(f"OpenAI Error: {e}")
        return None

In [ ]:
def insert_into_database(tissue_name, cell_name, gene_name, source_urls, certainty):
    conn = psycopg2.connect(os.getenv("DATABASE_URL"))
    cur = conn.cursor()

    try:
        # Lookup IDs
        cur.execute("SELECT uberon FROM tissue WHERE tissuename = %s", (tissue_name,))
        uberon = cur.fetchone()
        if not uberon:
            raise ValueError(f"Tissue '{tissue_name}' not found.")
        uberon = uberon[0]

        cur.execute("SELECT cl FROM cell WHERE cellname = %s", (cell_name,))
        cl = cur.fetchone()
        if not cl:
            raise ValueError(f"Cell '{cell_name}' not found.")
        cl = cl[0]

        cur.execute("SELECT ensembl FROM gene WHERE genename = %s", (gene_name,))
        ensembl = cur.fetchone()
        if not ensembl:
            raise ValueError(f"Gene '{gene_name}' not found.")
        ensembl = ensembl[0]

        # Insert or update gene_expression
        cur.execute("""
            INSERT INTO gene_expression (uberon, cl, ensembl, relationcertainty)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (uberon, cl, ensembl)
            DO UPDATE SET relationcertainty = EXCLUDED.relationcertainty
            RETURNING expressionid
        """, (uberon, cl, ensembl, certainty))
        expressionid = cur.fetchone()[0]

        # Insert sources and validate links
        for url in source_urls:
            # Check or insert source
            cur.execute("SELECT sourceid FROM source WHERE url = %s", (url,))
            row = cur.fetchone()
            if row:
                sourceid = row[0]
            else:
                cur.execute("INSERT INTO source (url) VALUES (%s) RETURNING sourceid", (url,))
                sourceid = cur.fetchone()[0]

            # Insert validate link if not exists
            cur.execute("""
                INSERT INTO validate (expressionid, sourceid)
                VALUES (%s, %s)
                ON CONFLICT DO NOTHING
            """, (expressionid, sourceid))

        conn.commit()
        print(f"✅ Inserted/Updated expressionid {expressionid} with sources.")

    except Exception as e:
        conn.rollback()
        print(f"❌ Error: {e}")

    finally:
        cur.close()
        conn.close()

In [ ]:
def validate_cell_gene_relation(tissue_name, cell_name, gene_name):
    print(f"\n[Validation Start] Processing: {tissue_name} - {cell_name} - {gene_name}")

    # Step 2: Check database first
    stored_status = check_database(tissue_name, cell_name, gene_name)
    if stored_status is not None:
        print(f"[Validation] Relation already exists in DB with status {stored_status}. Skipping process.")
        return float(stored_status)  # Ensure status is returned as a float

    # Step 3: Query Tavily
    try:
        sources = query_tavily(tissue_name, cell_name, gene_name)
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

    # Step 3a: If no sources found, set status to 0.0 and insert into database
    if not sources:
        status = 0.0
        print(f"[Validation] No references found in Tavily. Setting status to {status}.")
        insert_into_database(tissue_name, cell_name, gene_name, [], status)
        return float(status)  # Ensure status is a float

    # Step 4: Summarize with OpenAI
    status = summarize_with_openai(tissue_name, cell_name, gene_name, sources)
    status = float(status)  # Ensure status is a float
    print(f"[Validation] OpenAI summary status: {status}")

    # Step 5: Insert into Database
    insert_into_database(tissue_name, cell_name, gene_name, sources, status)
    print(f"[Validation] Inserted/Updated in DB with status: {status}")

    return status  # Return the validation score as a float

In [ ]:
validate_cell_gene_relation("skin of body", "neural cell", "MMP3")

In [ ]:
x_df = results.copy()
for cluster in x_df:
    unique_cells = x_df[cluster]['cell'].unique()[:10]
    x_df[cluster] = x_df[cluster][['gene', 'cell', 'cell score']][x_df[cluster]['cell'].isin(unique_cells)]

In [ ]:
display_dataframe_as_markdown(x_df[0])

In [ ]:
def main_data_validation(results, target_uberon_id):
    ex_df = results.copy()
    for cluster in ex_df:
        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['gene', 'cell', 'cell score']][ex_df[cluster]['cell'].isin(unique_cells)]
    tissue_name = ontl.tissue(target_uberon_id)

    for cluster in ex_df:
        ex_df[cluster]["certainty score"] = ex_df[cluster].apply(lambda row: validate_cell_gene_relation(tissue_name, row["cell"], row["gene"]), axis=1)

    return ex_df

In [ ]:
res = main_data_validation(results, TARGET_UBERON_ID)

In [ ]:
valid_df = res[0].copy()
mean_certainty = valid_df.groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
final_df = valid_df.merge(mean_certainty, on='cell')


unique_cells = final_df['cell'].unique()
final_df = final_df[['cell', 'cell score', 'cell certainty']][final_df['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)



In [ ]:
def export_validation_dataframe(results):
    ex_df = results.copy()
    for cluster in ex_df:
        mean_certainty = ex_df[cluster].groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
        ex_df[cluster] = ex_df[cluster].merge(mean_certainty, on='cell')

        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['cell', 'cell score', 'cell certainty']][ex_df[cluster]['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)
    return ex_df

In [ ]:
tes = export_validation_dataframe(res)[0]

In [ ]:
display_dataframe_as_markdown(tes)

In [ ]:
display_dataframe_as_markdown(final_df)

In [ ]:
display_dataframe_as_markdown(mean_certainty)

In [ ]:
for cluster in results:
    valids = {}
    print(f"\nProcessing Cluster: {cluster}")
    valids[cluster] = data_validation(results[cluster], tl.tl_uberon(TARGET_UBERON_ID)).reset_index(drop=True)

In [ ]:
display_dataframe_as_markdown(results[0])

# Analysis using OpenAI and Tavily

In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.runnables import RunnableConfig, chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import chain
from tavily import TavilyClient
import ast
import asyncio

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0
)

In [ ]:
tavily = TavilyClient()

In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [ ]:
tool = TavilySearchResults(
    max_results=10,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False,
    include_images=False,
)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.0
)

In [ ]:
res = tool.invoke('is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?')

In [ ]:
llm.invoke(f"answer with yes or no. based on this search results,\n{res}\n, is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?")

In [ ]:
tissue = tl.tl_uberon(TARGET_UBERON_ID)
tissue

In [ ]:
cell = final_df['cell'][0]
cell

In [ ]:
genes = [final_df['gene'][0]]
genes

In [ ]:
query = f"is {cell} of {tissue} tissue related to these genes: {genes}?"

In [ ]:
prompt = ChatPromptTemplate(query)

In [ ]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert in cellular biology. answer with a boolean array"),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [ ]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

In [ ]:
final_df.head(1)

In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent = create_conversational_retrieval_agent(
    llm,
    tools=[tavily_tool],
)

In [ ]:
def generate_queries(cell_name, gene_names: list):
    return f"answer with yes or no. Is {cell_name} related to this genes: {gene_names}?"

In [ ]:
cell_names = results[0]['cell'].unique()[:20].tolist()

In [ ]:
gene_names = results[0][results[0]['cell'] == cell_names[1]]['gene'].tolist()

In [ ]:
gene_names

In [ ]:
cell_names

In [ ]:
test = generate_queries(cell_names[1], gene_names)
test

In [ ]:
# run the agent
resp = agent.invoke(test)

In [ ]:
resp

In [ ]:
resp

# Analysis using LangChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv(override=True)

client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class cell(BaseModel):
        cell_type : str = Field(description="name of the chosen cell.")
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
        
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[cell]
  

In [ ]:
initial_df = df

In [ ]:
prompt = ChatPromptTemplate.from_template("""
                                          
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, outline the steps to create and apply a scoring system, then finally return a table of top 5 possible cell type identities for the cluster, and its calculated value.

Ensure that:
1. Describe each row of the dataset, and what it represents.
2. What metric is being used, and what value do they indicate.
3. The steps for scoring each possible cell type.
4. How to obtain the top five possible candidates.                                      
""")

chain = prompt | llm | StrOutputParser()

In [ ]:
parser = JsonOutputParser(pydantic_object=CellAnnotation)

prompt = PromptTemplate(
    template="""You are an expert in single-cell RNA sequencing.

        I have a cluster of cells with its genes as follow

        {initial_df}

        with the gene expression of each cell

        {expression_data}

        Based on the given data, which cell is the most valid to be annotated for this cluster of cells according to you?
        
        \n{format_instructions}""",
    input_variables=["initial_df", "expression_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [ ]:
response = chain.invoke({'initial_df': initial_df, 'expression_data': results[0].to_csv(sep='\t', index=False)})

In [ ]:
response

# Analysis using OpenAI

In [ ]:
# MODEL = "o3-mini"

In [ ]:
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [ ]:
initial_df = df

In [ ]:
instructions = """
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset. You will do this by creating and applying a scoring system.

The dataset is structured as follows:

Each row represents the cells that strongly express a top genes from the cluster. The columns are: "Gene" (a top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Follow these steps precisely to prepare the information, which will be automatically formatted into JSON:

**Step 1: Data Understanding**

*   Explain what each row in the dataset represents. Specifically, explain that each row shows a potential connection between the top gene from the cluster and one known cell type, and the metrics for THAT cell type, not for the cluster itself.
*   Explain the meaning of each column: "Gene", "Cell Name", "Expression Value", "Cell Percentage", "Cell Count", and "Tissue Composition." For each, explain what a higher or lower value might indicate *in the context of identifying similarity to the unannotated cluster*.

**Step 2: Develop a Scoring Function (CellTypeScore)**

*   Explain that the goal is to assign each known "Cell Name" (cell type) a score that reflects its similarity to the unannotated cluster.
*   Propose a formula for calculating a `GeneScore` for each *Gene* within each *Cell Name*. This formula should incorporate both "Cell Percentage" and "Expression Value". Justify why you chose to include these two metrics and how they are combined.
*   Propose a formula for combining the `GeneScore` values across all the `Gene` entries for a given "Cell Name" to calculate a final overall `CellTypeScore` for that cell type.

**Step 3: Calculate CellTypeScore for All Cell Names**

*   Explain that the goal is to now apply your scoring system to the known "Cell Name" values and find out which ones are the highest, with relation to what each Gene expresses.
*   Describe in precise steps what it should do: go through each unique "Cell Name" in the dataset, calculate a `GeneScore` and `CellTypeScore` based on Gene, Expression Value, and the percentage that are expressed.

**Step 4: Identify Top 5 Candidates**

*   Explain how to identify the top 5 candidates from the `CellTypeScore` values, now that they have been calculated in Step 3.
"""

In [ ]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster.

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Understand each row of the data and what it represents.
3. Prioritize cell types that's expressing more genes from the cluster, then consider the cell count, cell percentage, and expression value.
4. The steps for determining each possible cell type.
5. How to obtain the top three possible candidates.
6. give the explanation as if writing a report
"""

In [ ]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:
Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).
Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster. The dataset does not contain any representation of the unannotated cell cluster.

Cell similiarity is determined by the number of genes expressed in the cell then the cell count, cell percentage, and expression value. The more genes expressed in a cell, the higher the cell count, cell percentage, and expression value, the more similar the cell is to the unannotated cell cluster. the number of genes expressed in the cell is the most important factor in determining the similarity of the cell to the unannotated cell cluster.

Follow these steps precisely to prepare the information
Step 1 : Read the whole dataset
Step 2 : Explain to me what do you think about each cell's similiarity to the unannotated cell cluster based on the given data.
Step 3 : Based on you analysis, what are the top 3 possible cell type identities for the cluster?

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Do not consider cells that expressing lesser amount of genes from the cluster.
3. Do not consider cells that express a gene with parameters that deviate too much from other genes expressed by that cell.
4. Explanations must be written like a report.
"""

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class top_cell(BaseModel):
        class gene(BaseModel):
            gene_name : str = Field(description="Name of the gene expressed by the cell type")
            metrics : str = Field(description="cell count, cell percentage, and expression value of the gene")
        
        cell_type : str = Field(description="name of the chosen cell.")
        gene_info : list[gene]
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
    
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[top_cell]

In [ ]:
print(results[0].to_csv(sep='\t', index=False))

In [ ]:
content = f"expression data : {results[23].drop(columns=['tissue composition']).to_csv(sep='\t', index=False)}"

In [ ]:
#normal
completion = client.beta.chat.completions.parse(
    model = 'gpt-4o-mini-2024-07-18',
    messages=[
        {"role": "developer", 
         "content": instructions},
        {"role": "user", 
         "content": content}
    ],
    temperature=0.0,
    response_format=CellAnnotation,
)

In [ ]:
#Reasoning Model
response = client.chat.completions.create(
    model = "o3-mini",
    reasoning_effort="medium",
    messages=[
        {"role": "developer", 
         "content": """
            - You are an expert in single-cell RNA sequencing. 
            - You will be provided with cell clustering data and gene expression data for each cell. 
            - Based on the given data, you need to identify the 4 most valid cell to be annotated for the cluster. 
            - Provide a list of cells in the order of validity and explain your reasoning step by step. only give out the name of the cells from the given data.
            - Analyze only the data provided. Do not reference or infer from any external sources
            - the answer is in the format of 
                {
                cells : [cell1, cell2, cell3, cell4],
                explanation : <explanation>
                }
            """},
        {"role": "user", 
         "content": content}
    ],
)

In [ ]:
print(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [ ]:
import json

output_dict = json.loads(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [ ]:
output_dict['cells']

# Color Test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample DataFrame
data = {'Values': [5, 3, 8, 2, 9, 1, 7, 4, 6, 5, 3, 2]}
df = pd.DataFrame(data)

# Define normalization (scale from min=1 to max=9)
norm = mcolors.Normalize(vmin=1, vmax=9)
# Choose a colormap (reversed for light→dark)
cmap = plt.get_cmap('YlGnBu_r')  # Or 'Blues_r', 'Greys_r', etc.

# Map values to hex color codes
df['Color'] = df['Values'].apply(
    lambda x: mcolors.to_hex(cmap(norm(x)))  # Convert to hex
)

# Display DataFrame with hex color codes
print(df)

# Test Marker

In [ ]:
import pandas as pd
import requests
def get_marker(tissue, cell):
   # URL to send the POST request to
    url = "https://api.cellxgene.cziscience.com/wmg/v2/markers"

    # Payload to send in the POST request
    payload = {
        "celltype": cell,
        "n_markers":50,
        "organism":"NCBITaxon:9606",
        "test":"ttest",
        "tissue":tissue
    }

    # Send POST request
    return requests.post(url, json=payload).json()['marker_genes']

In [ ]:
get_marker("UBERON:0001043", "cell CL:1000615")

# Test Function 2

In [ ]:
resultsv2 = main_data_analysisv2(ensembl_df, TARGET_UBERON_ID)

In [ ]:
display_dataframe_as_markdown(resultsv2[23])